In [24]:
import numpy as np
import h5py
from utils.DRP_uitls import *
import pyvista as pv

In [2]:
#Download binary image
file_url = "https://www.digitalrocksportal.org/projects/175/images/159992/download/"
download_file(file_url,'./data/3D_binary_MicrostructureData.mat')

#Download velocity data from DRP
file_url = "https://www.digitalrocksportal.org/projects/175/images/159989/download/"
download_file(file_url,'./data/VX.mat')
file_url = "https://www.digitalrocksportal.org/projects/175/images/159990/download/"
download_file(file_url,'./data/VY.mat')
file_url = "https://www.digitalrocksportal.org/projects/175/images/159991/download/"
download_file(file_url,'./data/VZ.mat')

'./data/VZ.mat'

In [7]:
#Load binay image data
arrays = {}
f = h5py.File('./data/3D_binary_MicrostructureData.mat')
for k, v in f.items():
    arrays[k] = np.array(v)
f.close()

img=arrays['Microstruct']
scale=arrays['scale'].flatten()[0]
print('Image Info',img.shape,'scale=',scale,'dimension= mm',np.array(img.shape)*scale*1e3)

Image Info (565, 525, 1071) scale= 1.6812865497076024e-05 dimension= mm [ 9.49926901  8.82675439 18.00657895]


In [22]:
#Load velocity data
Velocity = {}
for vfile in ['./data/VX.mat','./data/VY.mat','./data/VZ.mat']:
    f = h5py.File(vfile)
    for k, v in f.items():
        Velocity[k] = np.array(v)*1e6 #um/s
    f.close()
Velocity.pop('scale')

Vel_magn=np.sqrt(np.power(Velocity['VX'],2)+np.power(Velocity['VY'],2)+np.power(Velocity['VZ'],2))

In [10]:
#Convert numpy array to paraview vti image
NX,NY,NZ=1000,500,500 #Correct axis order (input data is reversed)
os.makedirs("./data", exist_ok=True)

vtkimg = pv.UniformGrid(np.array([NX,NY,NZ])+1)
vtkimg.cell_arrays['MetaImage']=np.einsum('ijk->kji', img[0:NZ,0:NY,0:NX]).flatten(order="F")
vtkimg.save("./data/image_binary.vti")

In [23]:
Vel_field=np.zeros([np.prod([NX,NY,NZ]),3],dtype=np.float32)
for i,v in enumerate(['VX','VY','VZ']):
    Vel_field[:,i]=np.einsum('ijk->kji', Velocity[v][0:NZ,0:NY,0:NX]).flatten(order="F")

vtkimg = pv.UniformGrid(np.array([NX,NY,NZ])+1)
vtkimg.cell_arrays['Velocity(um/s)']=Vel_field
vtkimg.save("./data/image_vel.vti")